In [1]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/30 22:20:43 WARN Utils: Your hostname, LAPTOP-MLKFF9UL resolves to a loopback address: 127.0.1.1; using 172.24.136.186 instead (on interface eth0)
24/03/30 22:20:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/mzfuadi/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mzfuadi/.ivy2/cache
The jars for the packages stored in: /home/mzfuadi/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d8b0f1de-0ca7-448b-b291-2d241550d7db;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: re

24/03/30 22:20:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/30 22:20:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/30 22:20:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

In [ ]:
query.stop()

In [1]:
from kafka import KafkaProducer
import pandas as pd
import time 

# Record the start time
start_time = time.time()

# Create a KafkaProducer instance
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

# Define the topic name
topic = 'green-trips'

# Define the data types for each column
dtype_dict = {
    'lpep_pickup_datetime': 'str',
    'lpep_dropoff_datetime': 'str',
    'PULocationID': 'int',
    'DOLocationID': 'int',
    'passenger_count': 'float',
    'trip_distance': 'float',
    'tip_amount': 'float'
}

# Define the columns you want to keep
columns_to_keep = [
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]

# Read the CSV file into a DataFrame, specifying the dtype parameter
df_green = pd.read_csv('dataset/green_tripdata_2019-10.csv', usecols=columns_to_keep, dtype=dtype_dict, na_values=[''])
df_zones = pd.read_csv('dataset/taxi_zone_lookup.csv', dtype={'LocationID':'int'})
columns_to_keep_zones = ['LocationID', 'Zone']
df_zones = df_zones[columns_to_keep_zones]
# Fill missing values in integer columns with zeros
df_green['passenger_count'] = df_green['passenger_count'].fillna(0).astype(int)

#df_green['lpep_pickup_datetime'] = pd.to_datetime(df_green['lpep_pickup_datetime'])
#df_green['lpep_dropoff_datetime'] = pd.to_datetime(df_green['lpep_dropoff_datetime'])
df_joined = df_green.join(df_zones, on='DOLocationID', how='inner')
df_joined = df_joined.drop('LocationID
df_joined

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount,LocationID,Zone
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1,5.88,0.00,197,Richmond Hill
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1,0.80,0.00,264,NaN
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2,7.50,0.00,229,Sutton Place/Turtle Bay North
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1,0.90,0.00,182,Parkchester
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1,2.52,2.26,189,Prospect Heights
...,...,...,...,...,...,...,...,...,...
476381,2019-10-31 23:30:00,2019-11-01 00:00:00,65,102,0,7.04,0.00,103,Governor's Island/Ellis Island/Liberty Island
476382,2019-10-31 23:03:00,2019-10-31 23:24:00,129,136,0,0.00,0.00,137,Kips Bay
476383,2019-10-31 23:02:00,2019-10-31 23:23:00,61,222,0,3.90,0.00,223,Steinway
476384,2019-10-31 23:42:00,2019-10-31 23:56:00,76,39,0,3.08,0.00,40,Carroll Gardens
